In [1]:
# !pip install -r requirements.txt

import pandas as pd
import os
from google.cloud import storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 19.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 6.7 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.1 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 259.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 51.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 21.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# Setup

In [2]:
file_prefix = "right_left_up_down_1200"
data_folder = "data_" + file_prefix
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gcsaccesskey.json"


if "data_" + file_prefix not in os.listdir():
    os.mkdir(data_folder)
    os.mkdir(data_folder + "/train")
    os.mkdir(data_folder + "/test")
    os.mkdir(data_folder + "/validation")

    # Create a client
    client = storage.Client()

    labels_bucket = "something-somethingv2-labels"
    labels_bucket = client.bucket(labels_bucket)


    labels_bucket.blob(f"{file_prefix}_indx_df.csv").download_to_filename(data_folder + f"/indx_df.csv")

In [3]:
indx_df = pd.read_csv(f"{data_folder}/indx_df.csv")

indx_df

,category,type,name
0,Pushing something from left to right,train,102893.avi
1,Pushing something from left to right,train,128933.avi
2,Pushing something from left to right,train,200719.avi
3,Pushing something from left to right,train,31085.avi
4,Pushing something from left to right,train,58685.avi
...,...,...,...
4795,Moving something down,validation,95480.avi
4796,Moving something down,validation,154757.avi
4797,Moving something down,validation,203415.avi
4798,Moving something down,validation,159730.avi


## Download avi files from gcs

In [11]:
train_file_ids = list(indx_df[indx_df["type"] == "train"]["name"])
test_file_ids = list(indx_df[indx_df["type"] == "test"]["name"])
validation_file_ids = list(indx_df[indx_df["type"] == "validation"]["name"])

def download_files_to_folder(folder_prefix, type, file_ids):
    # Create a client
    client = storage.Client()

    bucket = "something-somethingv2-avi"
    bucket = client.bucket(bucket)

    for f_id in file_ids:
        bucket.blob(f"{f_id}").download_to_filename(f"{folder_prefix}/{type}/{f_id}")

download_files_to_folder(data_folder, "train", train_file_ids)
download_files_to_folder(data_folder, "test", test_file_ids)
download_files_to_folder(data_folder, "validation", validation_file_ids)

## Validate Files

In [4]:
from validate_files import validate_files_downloaded

validate_files_downloaded(data_folder)

## Validate Videos

In [5]:
from validate_files import check_folder_is_working

check_folder_is_working(data_folder, "test")
check_folder_is_working(data_folder, "validation")
check_folder_is_working(data_folder, "train")